## Задание 1

**Написать теггер на данных с руским языком**

## загрузка данных

In [1]:
#!pip install pyconll

In [2]:
import pyconll

In [3]:
import nltk
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger, BigramTagger, TrigramTagger
import numpy as np
from scipy.sparse import hstack

In [4]:
#!mkdir datasets

In [5]:
# !wget -O ./datasets/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
# !wget -O ./datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

In [6]:
full_train = pyconll.load_from_file('datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('datasets/ru_syntagrus-ud-dev.conllu')

In [7]:
# for sent in full_train[1:2]:
#     for token in sent:
#         print(token.form, ' _ ', token.upos)
#     print()

In [8]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form.lower(), token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form.lower(), token.upos) for token in sent])
    
fdata_sent_train = []
for sent in full_train[:]:
    fdata_sent_train.append([token.form.lower() for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form.lower() for token in sent])

### 1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации 

In [9]:
unigram_tagger = UnigramTagger(fdata_train)
unigram_tagger.evaluate(fdata_test)

0.8819465507363596

In [10]:
bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
bigram_tagger.evaluate(fdata_test)

0.8883075523202911

In [11]:
trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
trigram_tagger.evaluate(fdata_test)

0.8874734607218684

In [12]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NN') 
tag = backoff_tagger(fdata_test,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
tag.evaluate(fdata_test) 

0.9838826542648199

<b>
    Подозрительно высокие результаты гораздо выше чем в методичке, посмотрим распределение тегов
</b>

In [13]:
fdata_pairs_test = []
for sent in fdata_test[:]:
    fdata_pairs_test.extend(i for i in sent)

tags = [tag for (word, tag) in fdata_pairs_test]

In [14]:
nltk.FreqDist(tags)

FreqDist({'NOUN': 27974, 'PUNCT': 22694, 'VERB': 13078, 'ADJ': 11222, 'ADP': 10585, 'ADV': 6165, 'PRON': 5598, 'PROPN': 4438, 'CCONJ': 4410, 'PART': 3877, ...})

<b>
    Всё перепроверил. Просто в методичке мы сразу вызвали bigram tagger, а в качестве backoff указали юниграм, обученный на другом датасете :)
</b>

### 2. написать свой теггер как на занятии, но улучшить попробовать разные векторайзеры, добавить знание не только букв и слов но и совместно объединить эти признаки

In [15]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [16]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [17]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
test_enc_labels = le.transform(test_label)

In [18]:
vectorizer = CountVectorizer(ngram_range=(1, 3), analyzer='char', max_features=30)

In [19]:
X_train = vectorizer.fit_transform(train_tok)

In [20]:
X_test = vectorizer.transform(test_tok)

In [21]:
lr = LogisticRegression(random_state=0, max_iter=10)
lr.fit(X_train, train_enc_labels)

C:\Users\Pavel\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(max_iter=10, random_state=0)

In [22]:
pred = lr.predict(X_test)

In [23]:
accuracy_score(test_enc_labels, pred)

0.6128972466552084

<b>
    0.612 - скор для бейзлайна (держим в голове, что у нас всего 30 признаков)
</b>

In [24]:
word_vectorizer = CountVectorizer(ngram_range=(1,5), analyzer='word', max_features=30)

In [25]:
X_train_word = word_vectorizer.fit_transform(train_tok)
X_test_word = word_vectorizer.transform(test_tok)

In [26]:
lr1 = LogisticRegression(random_state=0, max_iter=10)
lr1.fit(X_train_word, train_enc_labels)
pred1 = lr1.predict(X_test_word)
accuracy_score(test_enc_labels, pred1)

C:\Users\Pavel\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.3118070299598962

<b>
    анализировать только по словам не очень, попробуем объединить два подхода <br>
    При использовании только vectorizer = CountVectorizer(ngram_range=(1, 9), analyzer='char') скор 0.79
</b>

In [27]:
vectorizer = CountVectorizer(ngram_range=(1, 9), analyzer='char')
X_train_c = vectorizer.fit_transform(train_tok)
X_test_c = vectorizer.transform(test_tok)

In [28]:
word_vectorizer = CountVectorizer(ngram_range=(1,5), analyzer='word')
X_train_w = word_vectorizer.fit_transform(train_tok)
X_test_w = word_vectorizer.transform(test_tok)

In [29]:
X_train = hstack((X_train_c, X_train_w))
X_test = hstack((X_test_c, X_test_w))

In [30]:
X_train.shape, X_train_c.shape, X_train_w.shape

((871526, 692910), (871526, 593425), (871526, 99485))

In [31]:
lr = LogisticRegression(random_state=0, max_iter=25)
lr.fit(X_train, train_enc_labels)
pred = lr.predict(X_test)
accuracy_score(test_enc_labels, pred)

C:\Users\Pavel\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.9126647120277693

### 3. вместо векторайзеров взять эмбединги попробовать (word2vec и fasttext по желанию дополнительно можно взять tf.keras.layers.Embedding)  

In [32]:
import gensim

C:\Users\Pavel\Anaconda3\envs\tensorflow\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [33]:
train_word2vec = fdata_sent_train
train_word2vec.extend(fdata_sent_test)

In [34]:
w2v_model = gensim.models.Word2Vec(min_count=1, negative=20, sg=1, window=2)
w2v_model.build_vocab(train_word2vec)
w2v_model.train(train_word2vec, epochs=5, total_examples=w2v_model.corpus_count)

(3850133, 4951090)

In [35]:
X_train_w2v = [w2v_model.wv[word] for word in train_tok]
X_test_w2v = [w2v_model.wv[word] for word in test_tok]

In [36]:
lr = LogisticRegression(random_state=0, max_iter=25)
lr.fit(X_train_w2v, train_enc_labels)
pred = lr.predict(X_test_w2v)
accuracy_score(test_enc_labels, pred)

C:\Users\Pavel\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7433019917096351

<b>
0.74 приемлимый скор для минималистичной модели (эмбединги учили сами на маленьком корпусе)
</b>

In [37]:
ft_model = gensim.models.FastText(train_word2vec, min_count=1, negative=20, sg=1, window=2)

In [38]:
X_train_ft = [ft_model.wv[word] for word in train_tok]
X_test_ft = [ft_model.wv[word] for word in test_tok]

In [39]:
lr = LogisticRegression(random_state=0, max_iter=25)
lr.fit(X_train_ft, train_enc_labels)
pred = lr.predict(X_test_ft)
accuracy_score(test_enc_labels, pred)

C:\Users\Pavel\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8639251171098304

### 4. взять не только эмбединги каждого слова, но и взять соседей, т.е. информацию о соседях количество соседей выбрать самим (узнать наилучшее количество соседей)   

In [115]:
import warnings
warnings.filterwarnings("ignore")

In [117]:
windows = [1,2,3,4,5,6,7,8,9]
for i in windows:
    ft_model = gensim.models.FastText(train_word2vec, min_count=1, negative=20, sg=1, window=i)
    X_train_ft = [ft_model.wv[word] for word in train_tok]
    X_test_ft = [ft_model.wv[word] for word in test_tok]
    lr.fit(X_train_ft, train_enc_labels)
    pred = lr.predict(X_test_ft)
    print(i, accuracy_score(test_enc_labels, pred))

1 0.8631837023556769
2 0.8636470865770229
3 0.861734573518013
4 0.8586762376571294
5 0.8587436389984161
6 0.855036565227648
7 0.8500320156371112
8 0.8508324065648907
9 0.8516159471573485


### 5. сравнить все реализованные методы сделать выводы

<b>
Лучше всего отработали комбинации теггеров. <br>
На самом деле все подходы можно считать работающими, просто результаты получились не идеальными, потому что для обучения мы использовали небольшой корпус.
</b>

# Задание 2

много дополнительных датасетов на русском языке

https://natasha.github.io/corus/  
https://github.com/natasha/corus

мы будем использовать данные http://ai-center.botik.ru/Airec/index.php/ru/collections/28-persons-1000

**Проверить насколько хорошо работает NER**

1. взять нер из nltk
2. проверить deeppavlov
3. написать свой нер попробовать разные подходы (с доп информацией без) так же с учётом соседей и без них
4. сделать выводы по вашим экспериментам какой из подходов успешнее справляется

при обучении своего нера незабудьте разделить выборку

In [1]:
!pip install corus

You should consider upgrading via the 'c:\users\pavel\anaconda3\envs\tensorflow\python.exe -m pip install --upgrade pip' command.


In [2]:
import corus

In [71]:
path = 'Persons-1000.zip'
records = corus.persons.load_persons(path)
rec = next(records)

In [78]:
rec = next(records)

In [79]:
rec

PersonsMarkup(
    text='Комиссар СЕ критикует ограничительную политику в отношении беженцев в европейских странах\r\n\r\n05/08/2008 10:32\r\n\r\nМОСКВА, 5 августа /Новости-Грузия/.  Проводимая в европейских странах ограничительная политика в отношении беженцев нарушает ряд международных стандартов, в частности, право на воссоединение семей, заявляет Комиссар Совета Европы по правам человека Томас Хаммарберг (Thomas Hammarberg) в размещенном на его сайте еженедельном комментарии.\r\n\r\n"Ограничительная политика в отношении беженцев в европейских странах уменьшает возможности воссоединения разделенных семей", - полагает он.\r\n\r\nПо сообщению РИА Новости, Хаммарберг констатирует, что в последнее время "правительства попытались ограничить приезд близких родственников к тем беженцам, которые уже проживают в стране".\r\n\r\nКомиссар не называет конкретных стран, одновременно отмечая, что в ряде случаев подобная линия привела "к неоправданным человеческим страданиям, когда члены семьи, за

In [83]:
rec.text

'Комиссар СЕ критикует ограничительную политику в отношении беженцев в европейских странах\r\n\r\n05/08/2008 10:32\r\n\r\nМОСКВА, 5 августа /Новости-Грузия/.  Проводимая в европейских странах ограничительная политика в отношении беженцев нарушает ряд международных стандартов, в частности, право на воссоединение семей, заявляет Комиссар Совета Европы по правам человека Томас Хаммарберг (Thomas Hammarberg) в размещенном на его сайте еженедельном комментарии.\r\n\r\n"Ограничительная политика в отношении беженцев в европейских странах уменьшает возможности воссоединения разделенных семей", - полагает он.\r\n\r\nПо сообщению РИА Новости, Хаммарберг констатирует, что в последнее время "правительства попытались ограничить приезд близких родственников к тем беженцам, которые уже проживают в стране".\r\n\r\nКомиссар не называет конкретных стран, одновременно отмечая, что в ряде случаев подобная линия привела "к неоправданным человеческим страданиям, когда члены семьи, зависящие друг от друга, о

In [84]:
rec.spans

[PersonsSpan(
     id=1,
     start=362,
     stop=398,
     value='ТОМАС ХАММАРБЕРГ (THOMAS HAMMARBERG)'
 ),
 PersonsSpan(
     id=2,
     start=624,
     stop=634,
     value='ХАММАРБЕРГ'
 )]

### 1. взять нер из nltk

In [85]:
from nltk import word_tokenize, pos_tag, ne_chunk

In [86]:
# nltk.download('maxent_ne_chunker')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('words')

In [87]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(rec.text))) if hasattr(chunk, 'label') }

{('Thomas Hammarberg', 'PERSON'),
 ('Комиссар', 'PERSON'),
 ('МОСКВА', 'ORGANIZATION'),
 ('РИА Новости', 'ORGANIZATION'),
 ('СЕ', 'ORGANIZATION'),
 ('Совета Европы', 'PERSON'),
 ('Хаммарберг', 'PERSON')}

<b>
Ошибочно попал "совет европы". Комиссар вполне сойдет за персону
</b>

### 3. написать свой нер попробовать разные подходы (с доп информацией без) так же с учётом соседей и без них

<b>
не успеваю нормально доделать до дедлайна. Оценку можно ставить за то, что есть
</b>

In [88]:
rec

PersonsMarkup(
    text='Комиссар СЕ критикует ограничительную политику в отношении беженцев в европейских странах\r\n\r\n05/08/2008 10:32\r\n\r\nМОСКВА, 5 августа /Новости-Грузия/.  Проводимая в европейских странах ограничительная политика в отношении беженцев нарушает ряд международных стандартов, в частности, право на воссоединение семей, заявляет Комиссар Совета Европы по правам человека Томас Хаммарберг (Thomas Hammarberg) в размещенном на его сайте еженедельном комментарии.\r\n\r\n"Ограничительная политика в отношении беженцев в европейских странах уменьшает возможности воссоединения разделенных семей", - полагает он.\r\n\r\nПо сообщению РИА Новости, Хаммарберг констатирует, что в последнее время "правительства попытались ограничить приезд близких родственников к тем беженцам, которые уже проживают в стране".\r\n\r\nКомиссар не называет конкретных стран, одновременно отмечая, что в ряде случаев подобная линия привела "к неоправданным человеческим страданиям, когда члены семьи, за

In [89]:
rec.text[362:398]

'Томас Хаммарберг (Thomas Hammarberg)'

['томас', 'хаммарберг', '(', 'thomas', 'hammarberg', ')']
['хаммарберг']


In [ ]:
# # установка deeppavlov

# !pip uninstall -y tensorflow tensorflow-gpu
# !pip install numpy scipy librosa unidecode inflect librosa transformers
# !pip install deeppavlov

In [ ]:
#!python -m deeppavlov install squad_bert
#!python -m deeppavlov install ner_ontonotes

In [ ]:
import deeppavlov
from deeppavlov import configs, build_model

In [ ]:
deeppavlov_ner = build_model(configs.ner.ner_bert_ent_and_type_rus, download=True)
rus_document = "Нью-Йорк, США, 30 апреля 2020, 01:01 — REGNUM В администрации президента США Дональда Трампа планируют пройти все этапы создания вакцины от коронавируса в ускоренном темпе и выпустить 100 млн доз до конца 2020 года, передаёт агентство Bloomberg со ссылкой на осведомлённые источники"
deeppavlov_ner([rus_document])